## Installing Transformers library

In [23]:
#Installing transformer package 
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Rating Estimation by ChatGPT

In [ ]:
!pip install openai

In [ ]:
import os 
import openai
os.environ["OpenAI_API_Key"] = ""
api_key = os.environ.get("OpenAI_API_Key")
openai.api_key = api_key

In [ ]:
# Loss to be added in the custom loss
def get_chatgpt_rating(prompt, sample):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", 
     "content": f"{prompt} {sample}"}
  ])
  return 10 - int(completion.choices[0].message)

In [ ]:
# Different prompts tested in order to generate a sensible rating 
prompt1 = "Provide me rating between 0 and 10 (without any explanation), where 0 is the best and 10 is the worst, for the following story summary: " 
prompt2 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response):"
prompt3 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response). The rating should be based on writing style, coherence and capture strength. Summary:"
prompt4 = "Assign a rating between 0 and 10 to the given artificial story summary. Only give rating as the response (no reasoning). The rating should be based on writing style, coherence, and capture strength. Summary:" # Best
prompt5 = "Provide me rating between 0 and 10 (without any explanation),  for the following story summary:" 

## Data Tokenization/Encoding

In [24]:
#Loading the standard T5 small model and tokenizer 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Can try different T5 models such as T5-large, T5-3B, T5-11B
base_tokenizer = AutoTokenizer.from_pretrained('t5-small')
base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [94]:
# Encoding the sequences
def encode_sequences(x, base_tokenizer = base_tokenizer):
  # try:
    # Input consists of different aspects of the story on which the output will be conditioned
    input = x['Input']
    # Label is the conditioned output - Story
    label = x['Summary']
    # Max length of the input sequence in T5 is 512 tokens (BART could be used for longer sequences - 1024 max length limit) 
    model_input = base_tokenizer(input, max_length = 512, truncation=True, padding='max_length')
    model_input['labels'] = base_tokenizer(label, max_length = 512, truncation=True, padding='max_length')['input_ids']
    return model_input

In [ ]:
encode_sequences(train_df.iloc[0])

In [95]:
# Loading the final dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/Training_Dataset')

In [97]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.3)

In [98]:
# Tokenizing the dataset
train_df = train_df.apply(encode_sequences, axis=1)
test_df = test_df.apply(encode_sequences, axis = 1)

In [99]:
train_df.dropna(inplace = True)
test_df.dropna(inplace = True)

In [100]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

## T5 Training Setup (Includes Custom Loss Function)

In [159]:
# Initializing the Data Collator for batching of the dataset
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
        tokenizer=base_tokenizer,
        return_tensors="pt",
        max_length = 512,
        padding = 'max_length',
        model = base_model
    )

In [ ]:
!pip install accelerate

In [160]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Path where model training loss and intermediate weights will be stored
model_path = f'/content/drive/MyDrive/Visual Story Telling/Story_Gen_Model'

#Specifying the training argument 
training_args = Seq2SeqTrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=2, 
    overwrite_output_dir = True, 
    evaluation_strategy="no", 
    gradient_accumulation_steps=8, 
    num_train_epochs=1,
    weight_decay=0.01, 
    lr_scheduler_type="cosine",
    learning_rate=5e-4, 
    fp16=True 
)

In [156]:
# Overwrite the Trainer API for utilizing custom loss function 
import torch
import torch.nn.functional as F
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
  def compute_loss(self, model, inputs, return_outputs=False):
      labels = inputs.pop("labels")
      input_ids = inputs.pop("input_ids")
      attention_mask = inputs.pop("attention_mask")
      outputs = model(input_ids = input_ids, labels = labels, attention_mask = attention_mask)
      loss_cross_entropy = outputs.loss
      logits = outputs.logits
      loss = custom_loss_function(logits) 
      return (loss_cross_entropy + torch.tensor(loss)).item()

# Check whether the tokenizer is being passed as an argument 
def custom_loss_function(logits, tokenizer=base_tokenizer):
    loss_GPT = 0
    summaries = base_tokenizer.batch_decode(F.softmax(logits, dim=-1).argmax(dim=-1), skip_special_tokens=True)
    for summary in summaries:
      loss_GPT = loss_GPT + (10 - int(get_chatgpt_rating(prompt4, summary)))
    return loss_GPT


In [161]:
# Initializing the trainer

trainer = Seq2SeqTrainer(
    model=base_model,                         # the instantiated  Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=train_df,       # training dataset
    eval_dataset = test_df,
)

In [ ]:
#Custom Trainer function using Customized Loss function 

trainer = CustomSeq2SeqTrainer(
    model=base_model,                         # the instantiated  Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=train_df,       # training dataset
    eval_dataset = test_df,
)

## Model Training 

In [ ]:
# Starting the training
trainer.train()

In [ ]:
# Saving the final model
trainer.save_model()

## Model Evaluation (Perplexity)

In [ ]:
trainer.evaluate()

## Model Eval (BLEU)

In [ ]:
!pip install evaluate

In [ ]:
from evaluate import load
#Loading the BLEU score metric
bleu = load("bleu")

In [ ]:
# Code to get BLEU Score rating of the model on test dataset 

"""
Refer:

1) https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt
2) https://huggingface.co/docs/evaluate/choosing_a_metric

"""

In [ ]:
# Get the samples
# Tokenize them
# Send to the model 
# Create pipeline or use generate function with different decoding algos
# Store the gold label and the generated ones in a list 
# Get the bleu score with n-gram similarities